```{contents}
```

## Embedding Versioning

**Embedding Versioning** means tracking and managing which embedding model was used to generate each embedding vector.
It ensures that:

```
query_embedding (model_version_X)
is compared with
document_embeddings (model_version_X)
```

NOT with a different version.

This prevents **retrieval failures**, **semantic mismatch**, and **hallucinations** in RAG.

---

### Why Embedding Versioning Is Required

#### Embedding models often change over time

Example:

* OpenAI updates an embedding model
* HuggingFace releases a new checkpoint
* Tokenization rules change
* Quantization affects scale

These updates alter:

* vector distributions
* dimensionality
* similarity behavior
* clustering structure

If your corpus was indexed earlier, embeddings become **incompatible**.

---

#### Without versioning, RAG silently breaks

Your RAG system might do this:

```
Documents embedded using:
   "text-embedding-3-small" (old version)

Queries encoded using:
   "text-embedding-3-small" (new version)
```

Even though the *model name is identical*, the embedding **space is different**.

Then cosine similarity becomes meaningless.
Retrieval returns irrelevant documents → LLM hallucinates.

This is a **silent, dangerous failure**.

---

###  What Problems Embedding Versioning Solves

#### ✔ Prevents mismatched embedding spaces

#### ✔ Ensures consistency across ingestion and query layers

#### ✔ Allows safe model upgrades

#### ✔ Enables re-indexing workflows

#### ✔ Avoids silent RAG degradation

#### ✔ Tracks when an embedding index is stale

---

### What Must Be Versioned?

Each stored embedding must record:

#### **1. Embedding Model Name**

```
"text-embedding-3-small"
```

#### **2. Embedding Model Version / Hash**

Include a stable version identifier:

```
"text-embedding-3-small@v1.0"
model_sha256 = "a93e...103"
```

#### **3. Dimensionality**

Example:

```
dim = 1536
```

If dimension changes, old embeddings become invalid.

#### **4. Tokenizer Version**

Tokenizer changes alter embedding vectors even if the model is same.

#### **5. Normalization Method**

Normalized or not normalized:

```
unit_norm = True
```

---

### Where to Store the Version?

#### **Option 1: Metadata in Vector DB**

Example (Qdrant/Pinecone/Weaviate):

```json
{
  "id": "doc123",
  "vector": [...],
  "metadata": {
    "model_name": "text-embedding-3-large",
    "model_version": "v2",
    "dim": 3072,
    "normalized": true
  }
}
```

#### **Option 2: Embedding Registry**

A centralized service storing:

* model name
* version
* last update date
* corpus index version

Similar to:

```
Docker registry
Model registry
```

#### **Option 3: Index Versioning**

Store embeddings as:

```
index_v1/
index_v2/
```

Useful during upgrades.

---

### How to Use Embedding Versioning in Practice

#### ✔ Step 1: Pin embedding model versions

Never rely on default model names:

```
model="text-embedding-3-large@2024-01-15"
```

#### ✔ Step 2: Maintain metadata in vector DB

Every vector gets model version metadata.

#### ✔ Step 3: Re-embed corpus when model updates

Automated batch jobs:

```
if current_model_version != index_model_version:
    re-embed_all_documents()
```

#### ✔ Step 4: Validate new embeddings

* compare cosine similarity before/after
* measure distribution shifts
* run search A/B tests

#### ✔ Step 5: Roll out new index safely

Use:

* canary index
* parallel index
* blue/green deployment

---

### Example: Why Versioning Prevents Errors

Suppose your old index uses:

```
SBERT v1 (768-dim)
```

You upgrade to:

```
SBERT v2 (1024-dim)
```

If you **don’t** re-embed your documents:

* query embedding is 1024-dim
* stored embeddings are 768-dim
* vector DB errors or returns garbage

Versioning catches this mismatch **before** production fails.

---

### Mental Model

Think of embeddings like coordinates on a map.

If the map changes (due to a new model version), the old coordinates:

* point to wrong places
* retrieval becomes useless

Embedding versioning ensures:

> You always use the same map for indexing and retrieval.

---

**Final Summary**

| Concept                    | Explanation                                              |
| -------------------------- | -------------------------------------------------------- |
| **Embedding Versioning**   | Labeling embeddings with model version info              |
| **Purpose**                | Ensures query + document embeddings come from same model |
| **Why Needed**             | To avoid RAG degradation and silent failures             |
| **Track**                  | model name, version, dimension, tokenizer, normalization |
| **Fixes**                  | mismatched embedding spaces                              |
| **When You Must Re-embed** | whenever embedding model updates                         |